 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/prepro_ai_challenger_step2.ipynb#预处理二" data-toc-modified-id="预处理二-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>预处理二</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/prepro_ai_challenger_step2.ipynb#导入工具包，并选择GPU卡" data-toc-modified-id="导入工具包，并选择GPU卡-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>导入工具包，并选择GPU卡</a></span></li><li><span><a href="http://localhost:8888/notebooks/prepro_ai_challenger_step2.ipynb#默认参数设置" data-toc-modified-id="默认参数设置-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>默认参数设置</a></span></li><li><span><a href="http://localhost:8888/notebooks/prepro_ai_challenger_step2.ipynb#构建词典" data-toc-modified-id="构建词典-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>构建词典</a></span></li><li><span><a href="http://localhost:8888/notebooks/prepro_ai_challenger_step2.ipynb#assign_splits" data-toc-modified-id="assign_splits-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>assign_splits</a></span></li><li><span><a href="http://localhost:8888/notebooks/prepro_ai_challenger_step2.ipynb#L,-label_start_ix,-label_end_ix,-label_length-=-encode_captions(imgs,-params,-wtoi)" data-toc-modified-id="L,-label_start_ix,-label_end_ix,-label_length-=-encode_captions(imgs,-params,-wtoi)-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>L, label_start_ix, label_end_ix, label_length = encode_captions(imgs, params, wtoi)</a></span></li><li><span><a href="http://localhost:8888/notebooks/prepro_ai_challenger_step2.ipynb#使用resnet进行图片特征的提取" data-toc-modified-id="使用resnet进行图片特征的提取-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>使用resnet进行图片特征的提取</a></span></li><li><span><a href="http://localhost:8888/notebooks/prepro_ai_challenger_step2.ipynb#其他信息存储" data-toc-modified-id="其他信息存储-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>其他信息存储</a></span></li></ul></li></ul></div>

# 预处理二
这是Image_Caption第二个运行的代码，核心功能是从所有语句中构建字典,并对每个句子进行编码存储；对训练和测试图片采用resnet152进行图片特征的提取.

## 导入工具包，并选择GPU卡

In [ ]:
import os
import json
import argparse
from random import shuffle, seed
import string
# non-standard dependencies:
import h5py
import numpy as np
import torch
import torchvision.models as models
from torch.autograd import Variable
import skimage.io
from misc.resnet_utils import myResnet
import jieba
import misc.resnet as resnet
from torchvision import transforms as trn
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

## 默认参数设置
- input_json: 训练集和验证集的coco存储形式的数据的位置.
- num_val:验证图片的个数
- output_json: 输出数据的位置
- output_h5: 
- word_count_threshold: 词频的阈值,大于等于word_count_threshold的词我们默认选择删除

In [ ]:
class Paras(): 
    def __init__(self):
        self.input_json = './preprocessed_data/coco_ai_challenger_new_version.json'
        self.num_val = 30000 
        self.output_json = './preprocessed_data/coco_ai_challenger_talk.json'
        self.output_h5 = './preprocessed_data/coco_ai_challenger_talk'

        self.max_length = 64  
        self.images_root = '/home/jiangqy/EXP/competition/data/captions/' 
        self.word_count_threshold =  2
        self.num_test = 0

In [ ]:
params = Paras()

In [ ]:
imgs = json.load(open(params.input_json, 'r'))
imgs = imgs['images']
preprocess = trn.Compose([
    # trn.ToTensor(),
    trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

## 构建词典

**vocab,counts = build_vocab(imgs, params):该函数用来构建词典.**

- vocab为对应的词表,由输入的所有图像描述包含的词构成.
- counts为对应的词频(字典形式),形式为{词:出现次数}.
- count_thr用来控制vocab的数量,如果输入的图片中所有caption分词得到的集合中词的个数小于count_thr的,我们一律用UNK来表示.同时我们将每张图片对应的所有词(包含5个描述)进行存储.

In [ ]:
def build_vocab(imgs, params):
    count_thr = params.word_count_threshold #params.word_count_threshold 
 
    counts = {}
    for img in imgs:
        for sentence in img['sentences']:  
            for s in sentence['tokens']:
                    for w in s:
                        counts[w] = counts.get(w, 0) + 1 

    cw = sorted([(count, w) for w, count in counts.iteritems()], reverse=True)
    print 'top words and their counts:'
    # print '\n'.join(map(str, cw[:20]))
    for i in cw[:20]: 
        print(i[1])
        print(i[0]) 

        
    for i in cw[-20:]: 
        print(i[1])
        print(i[0]) 
    # print some stats
    total_words = sum(counts.itervalues())
    total_words
    print 'total words:', total_words
    print('num of words',len(counts.keys()))
    
    
    bad_words = [w for w, n in counts.iteritems() if n <= count_thr]
    vocab = [w for w, n in counts.iteritems() if n > count_thr]
    
    bad_count = sum(counts[w] for w in bad_words)
    print 'number of bad words: %d/%d = %.2f%%' % (len(bad_words), len(counts), len(bad_words) * 100.0 / len(counts))
    print 'number of words in vocab would be %d' % (len(vocab),)
    print 'number of UNKs: %d/%d = %.2f%%' % (bad_count, total_words, bad_count * 100.0 / total_words)

    # lets now produce the final annotations
    if bad_count > 0:
            # additional special UNK token we will use below to map infrequent words to
        print 'inserting the special UNK token'
        vocab.append('UNK')

    for img in imgs:
        img['final_captions'] = []
        caption = []
        for txt in img['sentences']:
            s_ = []
            for s in txt['tokens']: 
                for w in s:
                    if counts.get(w, 0) > count_thr:
                        s_.append(w)
                    else:
                        s_.append('UNK')
            caption.append(set(s_)) 
        img['final_captions'].append(caption)

    return vocab,counts

In [ ]:
vocab,counts = build_vocab(imgs, params)

- itow: {index: 词},index到词的映射
- wtoi: {词 : index}，词到index的映射

In [ ]:
itow = {i + 1: w for i, w in enumerate(vocab)}  # a 1-indexed vocab translation table
wtoi = {w: i + 1 for i, w in enumerate(vocab)}  # inverse table

## assign_splits
该函数就是将图片划分为train和split,标注该样本是训练集中的还是验证集中的样本,其实之前的就已经有了,这边多此一举.

In [ ]:
def assign_splits(imgs, params):
    num_val = params.num_val 
    count_val = 0
    count_train = 0
    
    for i,img in enumerate(imgs):
        if 'val' in img['filepath']:
            img['split'] = 'val'
            count_val += 1
        else:
            count_train += 1
            img['split'] = 'train'

    print 'assigned %d to train, %d to val.' % (count_train,count_val )

In [ ]:
assign_splits(imgs, params)

## L, label_start_ix, label_end_ix, label_length = encode_captions(imgs, params, wtoi)
- L:所有语句的编码,如果有1000张图片,每张图片对应5句话,那么L = 1000 $*$ 5, 每一行对应一个句子的编码.
- label_start_ix: 每张图片开始的位置句子开始的位置
- label_end_ix:每张图片结束的位置句子结束的位置
- label_length: 每个句子的长度

**<font color=red>label_start_ix[i]到label_end_ix[i]为第i张图片对应的句子部分.其编码为:  L[label_start_ix[i]:label_end_ix[i],:]</font>**

In [ ]:
max_length = params.max_length
N = len(imgs)
M = 0
#for img in imgs:
#    M = M + len(img['final_captions'])
M = sum(len(img['final_captions']) for img in imgs)  # total number of captions
print('Total number of captions:' + str(N))
print('Total number of captions:' + str(M))

In [ ]:
label_arrays = []
label_start_ix = np.zeros(N, dtype='uint32')  # note: these will be one-indexed
label_end_ix = np.zeros(N, dtype='uint32')
label_length = np.zeros(M, dtype='uint32')
caption_counter = 0
counter = 1

In [ ]:
def encode_captions(imgs, params, wtoi):
    """
    encode all captions into one large array, which will be 1-indexed.
    also produces label_start_ix and label_end_ix which store 1-indexed
    and inclusive (Lua-style) pointers to the first and last caption for
    each image in the dataset.
    """

    max_length = params.max_length
    N = len(imgs)
    M = 0
    #for img in imgs:
    #    M = M + len(img['final_captions'])
    M = sum(len(img['sentences'][0]['tokens']) for img in imgs)  # total number of captions
    print('Total number of images:' + str(N))
    print('Total number of captions:' + str(M))
    label_arrays = []
    label_start_ix = np.zeros(N, dtype='uint32')  # note: these will be one-indexed
    label_end_ix = np.zeros(N, dtype='uint32')
    label_length = np.zeros(M, dtype='uint32')
    caption_counter = 0
    counter = 1
    for i, img in enumerate(imgs):
        n = len(img['sentences'][0])
        assert n > 0, 'error: some image has no captions'

        Li = np.zeros((n, max_length), dtype='uint32')
        for j, sentence in enumerate(img['sentences'][0]['tokens']): 
#             for sentence in s['tokens']: 
            
            label_length[caption_counter] = min(max_length, len(sentence))  # record the length of this sequence
            caption_counter += 1

            for k, w in enumerate(sentence): 
                if k < max_length:
                    if w not in wtoi.keys():
                        Li[j, k] = wtoi['UNK']
                    else:
                        Li[j, k] = wtoi[w] 

        # note: word indices are 1-indexed, and captions are padded with zeros
        label_arrays.append(Li)
        label_start_ix[i] = counter
        label_end_ix[i] = counter + n - 1

        counter += n

    L = np.concatenate(label_arrays, axis=0)  # put all the labels together
    assert L.shape[0] == M, 'lengths don\'t match? that\'s weird'
    assert np.all(label_length > 0), 'error: some caption had no words?'

    print 'encoded captions to array of size ', `L.shape`
    return L, label_start_ix, label_end_ix, label_length 

L, label_start_ix, label_end_ix, label_length = encode_captions(imgs, params, wtoi)

In [ ]:
# encode captions in large arrays, ready to ship to hdf5 file
L, label_start_ix, label_end_ix, label_length = encode_captions(imgs, params, wtoi)

## 使用resnet进行图片特征的提取

导入resnet网络,进行图片特征的抽取,f_fc.h5,f_att.h5分别存储了对应图片的图像特征.

In [ ]:
resnet_type = 'resnet152'
if resnet_type == 'resnet101':
    resnet = resnet.resnet101(True)
    resnet.load_state_dict(torch.load('/home/jiangqy/PycharmProjects/Image_Caption/resnet/resnet101.pth'))
else:
    resnet = resnet.resnet152(True)
    resnet.load_state_dict(torch.load('/home/jiangqy/PycharmProjects/Image_Caption/resnet/resnet152.pth'))
my_resnet = myResnet(resnet)
my_resnet.cuda()
my_resnet.eval()

N = len(imgs)
f_lb = h5py.File(params.output_h5 + '_'+ resnet_type +'_label.h5', "w")
f_lb.create_dataset("labels", dtype='uint32', data=L)
f_lb.create_dataset("label_start_ix", dtype='uint32', data=label_start_ix)
f_lb.create_dataset("label_end_ix", dtype='uint32', data=label_end_ix)
f_lb.create_dataset("label_length", dtype='uint32', data=label_length)
f_lb.close()

In [ ]:
f_fc = h5py.File(params.output_h5 + '_'+ resnet_type +'_fc.h5', "w")
f_att = h5py.File(params.output_h5 + '_'+ resnet_type +'_att.h5', "w") 
dset_fc = f_fc.create_dataset("fc", (N, 2048), dtype='float32')
dset_att = f_att.create_dataset("att", (N, 20, 20, 2048), dtype='float32')

In [ ]:
for i, img in enumerate(imgs):
        # load the image
    I = skimage.io.imread(os.path.join(params.images_root, img['filepath']+'/'+img['filename']))
    # handle grayscale input images
    if len(I.shape) == 2:
        I = I[:, :, np.newaxis]
        I = np.concatenate((I, I, I), axis=2) 
    I = I.astype('float32') / 255.0
    I = torch.from_numpy(I.transpose([2, 0, 1])).cuda()
    I = Variable(preprocess(I), volatile=True)
    print('start:',I.size())
    tmp_fc, tmp_att = my_resnet(I)
    print('end:',I.size(),tmp_fc.size(),tmp_att.size())
    # write to h5
    dset_fc[i] = tmp_fc.data.cpu().float().numpy()
    dset_att[i] = tmp_att.data.cpu().float().numpy()
    if i % 1000 == 0:
        print 'processing %d/%d (%.2f%% done)' % (i, N, i * 100.0 / N)
f_fc.close()
f_att.close()

## 其他信息存储
** out[images] **
- 存储是否是train / val 的信息
- 存储图片的id以及图片对应的位置

** out[ix_to_word] **
- 下标到词的转换

In [ ]:
out = {}
out['ix_to_word'] = itow  # encode the (1-indexed) vocab
out['images'] = []

for i, img in enumerate(imgs):
    if i% 1000 == 0:
        print(i)
    jimg = {}
    jimg['split'] = img['split']
    if 'filepath' in img: jimg['file_path'] = img['filepath']  # copy it over, might need
    if 'cocoid' in img: jimg['id'] = img['cocoid']  # copy over & mantain an id, if present (e.g. coco ids, useful)

    out['images'].append(jimg)

json.dump(out, open(params.output_json, 'w'))